In [ ]:
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import json
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as model_selection
import sklearn.tree as tree
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import time
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import catboost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split


In [ ]:
df_train = pd.read_csv('Training_clean1.csv')
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

df_test = pd.read_csv('Test_clean1.csv')
    


In [ ]:
df_train['floor/stories'] = df_train.floor/df_train.stories
df_test['floor/stories'] = df_test.floor/df_test.stories

df_train['distance'] = np.sqrt((37.6 - df_train['longitude'])**2 + (55.75 - df_train['latitude'])**2)
df_test['distance'] = np.sqrt((37.6 - df_test['longitude'])**2 + (55.75 - df_test['latitude'])**2)

df_train.drop(['floor', 'stories', 'longitude', 'latitude'], axis=1, inplace=True)
df_test.drop(['floor', 'stories', 'longitude', 'latitude'], axis=1, inplace=True)

In [ ]:
#y1 = df_train['price']
X1 = df_train.drop(['price', 'id', 'address'], axis=1)
X1_test = df_test.drop(['id', 'address'], axis=1)
X1_test_area_total = X1_test.area_total.values
df_train['price/area'] = df_train['price']/df_train['area_total']
y1 = df_train['price/area']
X1_test

,area_total,area_kitchen,area_living,rooms,bathrooms_shared,bathrooms_private,windows_court,windows_street,phones,building_id,...,district,street,constructed,material,elevator_without,elevator_passenger,elevator_service,parking,floor/stories,distance
0,71.4,14.553146,39.194587,1,0,0,1,0,1,5148,...,0,2-я Черногрязская улица,2017,2,1,1,1,0,0.100000,0.052411
1,54.0,4.000000,50.000000,1,0,0,1,0,1,1290,...,11,улица Сервантеса,2017,2,0,1,1,0,0.266667,0.216436
2,39.4,19.600000,11.200000,1,1,0,1,0,1,681,...,11,Саларьево Парк ЖК,2021,3,0,1,1,1,1.111111,0.228879
3,60.9,12.412977,33.430677,2,1,0,0,1,1,4261,...,11,улица Александры Монаховой,2014,2,1,1,0,0,0.294118,0.217394
4,34.0,9.000000,17.000000,1,1,0,1,0,1,7530,...,11,улица 3-я Нововатутинская,2016,2,0,1,1,0,0.823529,0.345584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932,106.0,19.900000,56.700000,3,0,0,1,0,1,4252,...,8,улица Паршина,2020,2,0,1,1,0,0.800000,0.148935
9933,82.0,16.713697,45.013391,3,2,0,1,0,2,7380,...,7,Западный Порт ЖК,2021,2,0,1,1,1,0.150000,0.083756
9934,49.3,10.048601,27.062929,1,0,0,1,0,1,1480,...,7,Рублевское шоссе,2016,2,0,1,1,0,0.625000,0.225713
9935,38.8,10.500000,15.100000,1,1,0,0,1,2,2154,...,5,улица Архитектора Щусева,2019,2,1,1,1,0,1.000000,0.062356


In [ ]:
#y1 = df_train.price.values
#X1 = df_train.drop(['id','price', 'street', 'address'], axis=1)
#X1_test = df_test.drop(['id', 'street', 'address'], axis=1)
#categorical_features_indices = np.where(X1.dtypes != np.float64)[0]
#categorical_features_indices = np.where(X1_test.dtypes != np.float64)[0]
cat_feature_indices = np.array([3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X1, y1, test_size=0.2, random_state=1, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X1_test.shape


((18628, 21), (4657, 21), (18628,), (4657,), (9937, 21))

In [ ]:
# Drop are_total, price
# Kepp area_total as numpy arrays 

In [ ]:
area_total_train = X_train.area_total.values
area_total_val = X_val.area_total.values
X_train = X_train.drop(['area_total'], axis=1)
X_val = X_val.drop(['area_total'], axis=1)
X_train.head()

,area_kitchen,area_living,rooms,bathrooms_shared,bathrooms_private,windows_court,windows_street,phones,building_id,new,district,street,constructed,material,elevator_without,elevator_passenger,elevator_service,parking,floor/stories,distance
16628,6.00000,33.000000,2,0,1,1,1,1,674,0,8,улица Центральная,1977,3,1,0,0,1,0.800000,0.236988
19292,5.40000,20.000000,1,0,0,1,0,1,5284,0,2,улица Бочкова,1963,4,1,0,0,1,0.200000,0.067063
6194,7.25619,19.542399,1,1,0,1,0,1,8601,1,11,Сосенское поселение,2020,2,1,1,1,1,0.166667,0.228344
14066,9.00000,21.000000,1,0,1,1,0,1,6746,0,6,Старобитцевская улица,1992,3,0,1,1,1,0.785714,0.174128
6790,6.80000,48.900000,2,1,2,1,1,1,2500,1,1,Ленинградский проспект,2020,2,0,1,0,0,0.048780,0.069788


In [ ]:
#hyper parameter
n_estimators=[100,500,900,1100,1500]
max_depth=[2,3,5,10,15]
booster=["gbtree","gblinear"]
learning_rate="[0.05,0.1,0.15,0.20]"
min_child_weight=[1,2,3,4]
base_score=[0.25,0.5,0.75,1]

org_params = {
    'n_estimators': n_estimators,
    'learning_rate': learning_rate,
    "booster":booster,
    'eval_metric': 'RMSE',
    'use_best_model': True,
    'loss_function': 'RMSE',
    'max_depth': max_depth,
    "min_child_weight":min_child_weight,
    "base_score":base_score
}

In [ ]:
from catboost import CatBoostRegressor
#random_state=1, iterations=1500, depth=8, learning_rate=.1, loss_function='RMSE'
model = CatBoostRegressor(random_state=1, iterations=2000,cat_features = cat_feature_indices, depth=9, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train, y_train, eval_set=(X_val, y_val),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

1006:	learn: 23314.5972404	test: 74495.2002390	best: 74495.2002390 (1006)	total: 2m	remaining: 1m 58s
1007:	learn: 23305.6828956	test: 74496.1202954	best: 74495.2002390 (1006)	total: 2m	remaining: 1m 58s
1008:	learn: 23298.9816887	test: 74498.4551782	best: 74495.2002390 (1006)	total: 2m	remaining: 1m 58s
1009:	learn: 23288.1323131	test: 74495.9227270	best: 74495.2002390 (1006)	total: 2m	remaining: 1m 58s
1010:	learn: 23274.3003793	test: 74495.4046122	best: 74495.2002390 (1006)	total: 2m	remaining: 1m 58s
1011:	learn: 23261.3600623	test: 74487.9546894	best: 74487.9546894 (1011)	total: 2m 1s	remaining: 1m 58s
1012:	learn: 23244.5181148	test: 74482.3521852	best: 74482.3521852 (1012)	total: 2m 1s	remaining: 1m 58s
1013:	learn: 23233.6208420	test: 74478.0156516	best: 74478.0156516 (1013)	total: 2m 1s	remaining: 1m 57s
1014:	learn: 23230.0226570	test: 74478.7494537	best: 74478.0156516 (1013)	total: 2m 1s	remaining: 1m 57s
1015:	learn: 23225.0675892	test: 74478.0159658	best: 74478.0156516 (10

In [ ]:
print(model.score(X_train, y_train))
from sklearn.metrics import r2_score

y_pred = model.predict(X_val)
y_pred = y_pred.astype(float)
print(model.score(X_val, y_val)), print(r2_score(y_pred, model.predict(X_val)))

0.9598727405713643
0.8111237560278827
1.0


(None, None)

In [ ]:
df=pd.DataFrame({'Actual': y_val, 'Predicted':y_pred})
df

,Actual,Predicted
10307,641025.641026,628101.786351
2684,370277.777778,410751.350293
15931,213851.992410,183403.999139
10683,187598.116170,179337.453662
22217,189556.509299,460509.511372
...,...,...
5606,119453.752094,137121.613124
8243,500000.000000,772104.667444
13490,163109.756098,176523.018989
7008,179500.000000,164605.241542


In [ ]:
final_labels = model.predict(X1_test.drop(['area_total'], axis=1))
final_labels = final_labels.astype(float)
final_labels

array([411543.07217793, 177145.8460152 , 165676.5618941 , ...,
       215995.07915192, 249824.67965253, 102640.29838745])

In [ ]:

final_labels = final_labels * df_test.area_total.values
len(final_labels)

9937

In [ ]:
df_train.area_total.values

array([ 59.2,  88. ,  78.5, ...,  73.4, 100.8,  57.7])

In [ ]:
y_train_hat = model.predict(X_train) * area_total_train ## * edit
y_valid_hat = model.predict(X_val) * area_total_val ## * edit

y_train = y_train * area_total_train ## edit
y_val = y_val * area_total_val ## edit

print(f'Train RMSLE: {root_mean_squared_log_error(y_true=y_train, y_pred=y_train_hat) :.4f}')
print(f'Valid RMSLE: {root_mean_squared_log_error(y_true=y_val, y_pred=y_valid_hat) :.4f}')

#Train RMSLE: 0.1283
#Valid RMSLE: 0.1761

#Train RMSLE: 0.0489
#Valid RMSLE: 0.1451

Train RMSLE: 0.0987
Valid RMSLE: 0.1479


In [ ]:
submission = pd.DataFrame()
submission['id'] = df_test.id
submission['price_prediction'] = final_labels # Predict on non-nan entries
 # Fill missing entries with mean predictor
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
#submission.to_csv('sample_submission.csv', index=False)

submission.to_csv('sample_submission9.csv', index=False)

Generated 9937 predictions


In [ ]:
# SHAP analysis


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99c1e030-46bc-40fa-835c-0661eae0d488' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>